Based on the work found at
https://jovian.com/munimadhav/project-web-scraping-oscar-winning-movies-imdb-4ffed

### Extract the information and the data that we are trying to about the movies

**Before we build through the whole script - lets work thru the attributes we want and how to get those attributes with one movie from the container**

For the project, goal is to extract the following detail for the movies list:
- movie_id
- name 
- genre 
- year
- gross
- rating
- certificate (Public rating eg. R PG13)
- duration 
- rating_count
- oscar_wins (no longer app)
- oscar_nominations (no longer app)

-----

For each project, the following member association is desired
(found from https://www.imdb.com/title/{movie_id}/fullcredits)
- movie_id
- member_id
- association_type

### As I am specifically interested in the gender and racial representation on screen, I am focusing my interests in members in the following categories
- Directing
- Writing
- Casted
- Producing
- Stars (Found on movies page)
- Casting & Casting Department

-----

For each member, the following information is desired
- member_id
- name
- birth_year
- gender
- race
- first_movie_year
- last_movie_year
- total_movie_count



For the project, goal is to extract the following detail for the movies list:
- movie_id
```
 - found in <script type="application/ld+json"> - json - url
```
- url
```
 - found in <script type="application/ld+json"> - json - url
```
- name 
```
 - found in <script type="application/ld+json"> - json - name
```
- genre 
```
 - found in <script type="application/ld+json"> - json - genre
```
- year
```
 - found in <script type="application/ld+json"> - json - datePublished
```
- rating_count
```
 - found in <script type="application/ld+json"> - json - aggregateRating - ratingCount
```
- rating_value
```
 - found in <script type="application/ld+json"> - json - aggregateRating - ratingValue
```
- gross (May not be needed)
- certificate (Public rating eg. R PG13) (May not be needed)
- duration (May not be needed)
- oscar_wins (May not be needed, but if needed re-filtering from the search view can get the information)
- oscar_nominations (May not be needed, but if needed re-filtering from the search view can get the information)

-----
-----
Logic

Loop through the list of all oscar winning and nominated movies.

For each movie, iterate through the following
- Director
```
<h4 name="director" id="director" class="dataHeaderWithBorder">Directed by</h4>
```
- Writer
```
<h4 name="writer" id="writer" class="dataHeaderWithBorder">Writing Credits</h4>
```
- Cast
```
In <table class="cast_list">, between <td colspan="4" class="castlist_label"></td> and <td colspan="4" class="castlist_label">Rest of cast listed alphabetically:</td>
```
- Producing
```
<h4 name="producer" id="producer" class="dataHeaderWithBorder">
```
- Casting & Casting Department
```
<h4 name="casting_director" id="casting_director" class="dataHeaderWithBorder">
```
```
<h4 name="casting_department" id="casting_department" class="dataHeaderWithBorder">…</h4>
``` 



For each member, verify whether or not member exists in member datatable. If individual exists, move onto next individual.


## SEE IF INFORMATION CAN BE SCRAED FROM JSON AT 

If individual does not exit, add a new member data entry
- member_id
```
 - found in <script type="application/ld+json"> - json - url
 - found after https://www.imdb.com/name/
```
- url
```
 - found in <script type="application/ld+json"> - json - url
```
- name
```
 - found in <script type="application/ld+json"> - json - name
```
- birth_year
```
 - found in <script type="application/ld+json"> - json - birthDate
```
- job title (List)
```
 - found in <script type="application/ld+json"> - json - jobTitle
```
- gender
```
 - based on image found at <script type="application/ld+json"> - json - image, or below
 <div class="ipc-media ipc-media--poster-27x40 ipc-image-media-ratio--poster-27x40 ipc-media--baseAlt ipc-media--poster-s ipc-poster__poster-image ipc-media__img" style="width:100%">flex
 ```
- race
```
 - based on image found at <script type="application/ld+json"> - json - image, or below
 <div class="ipc-media ipc-media--poster-27x40 ipc-image-media-ratio--poster-27x40 ipc-media--baseAlt ipc-media--poster-s ipc-poster__poster-image ipc-media__img" style="width:100%">flex
 ```


Order of the looping logic

---

Start with search result of movie list (expansion into tv shows is tentative) (Starting with oscar nominees but may expand to all movies)

In the iteration of each movie, store the movie_id data into the 'movies' datatable

---

For each of the 'movies' iteration, enter the website and retrieve the rest of 'movies' datatable column data

---

Once 'movies' datatable is populated, run through the /fullcredits/ site in order to retrieve all associated members

For each individual listing, populate 'movie_member_associations' table
- Additional logic needs to handle duplicate member for same role
 - Actually, this may be as simple as running a data_table search and skipping if already exists

---

For each unique member_id in 'movie_member_association', enter the members' website to scrape and store data

In [12]:
import requests #packages that is used to download the content from web
import urllib # packages that is used to work with URL libraries
import requests #package built to make HTTP requests user friendly
import os # package used for file process
import re # package for regular expression - best to have it dont know if it is required or not
from bs4 import BeautifulSoup #a Python library for pulling data out of HTML and XML files
import pandas as pd # the omnipresent of all python to work with dataframes
requests.__version__ # check the version of the package

'2.31.0'

In [13]:
### Download the page using requests for 2017 Movies
#https://www.imdb.com/search/title/?title_type=feature,tv_movie&release_date=1900-01-01,2100-12-31&groups=oscar_winner,oscar_nominee&countries=us&sort=year,asc&count=10000
oscar_url = 'https://www.imdb.com/search/title/?release_date=2017&groups=oscar_winner' 
oscar_resp = requests.get(oscar_url) #get the content of the url from the request package.
print(oscar_resp.text[:200]) # look at the snapshot of the content



<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         

        <meta charset="utf-8">




        <script type="text/javascrip


In [14]:
oscar_soup = BeautifulSoup(oscar_resp.text, 'html.parser') # with the response text created - parse the html

In [15]:
oscar_containers = oscar_soup.find_all("div",{"class" : "lister-item mode-advanced"}) # get all the containers 
print('Containers is of type: ',type(oscar_containers)) # check the type 
print('Number of movies in the container:',len(oscar_containers)) # number of movies in the container

Containers is of type:  <class 'bs4.element.ResultSet'>
Number of movies in the container: 14


In [16]:

oscar_1 = oscar_containers[0] # get the first movie from the container
print('What is the data type of movie:',type(oscar_1)) # observe the tag of the first movie
print('details of the movie: ', len(oscar_1))

What is the data type of movie: <class 'bs4.element.Tag'>
details of the movie:  7


In [17]:
oscar_1

<div class="lister-item mode-advanced">
<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt1856101"></div>
</div>
<div class="lister-item-image float-left">
<a href="/title/tt1856101/"> <img alt="Blade Runner 2049" class="loadlate" data-tconst="tt1856101" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BNzA1Njg4NzYxOV5BMl5BanBnXkFtZTgwODk5NjU3MzI@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="67"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt1856101/">Blade Runner 2049</a>
<span class="lister-item-year text-muted unbold">(2017)</span>
</h3>
<p class="text-muted">
<span class="certificate">R</span>
<span class="ghost">|</span>
<span class="runtime">164 min</span>
<span class="ghost">|</span>
<span class="genre">
Action, Drama, Mystery            </span>
</p>
<di

In [18]:
oscar_1.div # div tag or a section or division within HTML

<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt1856101"></div>
</div>

In [19]:
oscar_1.a # a or anchor tag defines the href which is hypertext reference or link for the content on the page

<a href="/title/tt1856101/"> <img alt="Blade Runner 2049" class="loadlate" data-tconst="tt1856101" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BNzA1Njg4NzYxOV5BMl5BanBnXkFtZTgwODk5NjU3MzI@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="67"/>
</a>

In [20]:
oscar_1.h3 # access h3 content 

<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt1856101/">Blade Runner 2049</a>
<span class="lister-item-year text-muted unbold">(2017)</span>
</h3>

In [21]:
oscar_1.h3.a # access he anchor tag and now its getting close to get the movie title

<a href="/title/tt1856101/">Blade Runner 2049</a>

In [22]:
movie_name = oscar_1.h3.a.text # now access just the text part from the tag
movie_name

'Blade Runner 2049'

In [23]:
movie_year = oscar_1.h3.find('span', class_ = 'lister-item-year text-muted unbold').text
movie_year

'(2017)'

In [24]:
oscar_1.strong # attribute for rating seems to be pretty easy contained witin *strong* tag

<strong>8.0</strong>

In [25]:
# convert to float as expecting to see some decimal places for rating
movie_rating = float(oscar_1.strong.text) 
movie_rating

8.0

In [29]:
# now to get the rest attributes group in p tag with different class
oscar_1.find('span', class_ = 'certificate')

<span class="certificate">R</span>

In [28]:
movie_certificate = oscar_1.find('span', class_ = 'certificate').text
movie_certificate

'R'

In [30]:
def get_certificate(oscar):
    # cases where there is no certificate assigned - lets give default as Not Rated
    if oscar.find('span', class_ = 'certificate') is not None:
        cert = oscar.find('span', class_ = 'certificate').text
    else:
        cert = 'Not Rated'
    return cert

In [31]:
get_certificate(oscar_1)

'R'

In [32]:
movie_runtime = oscar_1.find('span', class_ = 'runtime').text
movie_runtime

'164 min'

In [33]:
movie_genre = oscar_1.find('span', class_ = 'genre').text
movie_genre

'\nAction, Drama, Mystery            '